**Astrophotography Field Catalog Agent**

This notebook demonstrates an Astrophotography Field Catalog Agent that automates the tedious, error-prone process of creating post-processing logs for my deep-sky imaging sessions. It serves as an autonomous virtual field assistant that transforms raw input (an image and a target name) into a complete, professional, and verifiable PDF log artifact in a single run. The  multi-agent system orchestrates external research (weather, science) and custom file processing, saving me dozens of hours per catalog entry.

**Astrophotography Field Catalog Agent**

This notebook demonstrates an Astrophotography Field Catalog Agent that automates the tedious, error-prone process of creating post-processing logs for my deep-sky imaging sessions. It serves as an autonomous virtual field assistant that transforms raw input (an image and a target name) into a complete, professional, and verifiable PDF log artifact in a single run. The  multi-agent system orchestrates external research (weather, science) and custom file processing, saving me dozens of hours per catalog entry.

**Problem Statement:**
I am an amateur Astrophotography enthusiast. Over the years I have amassed a collection of files from astrophotography trips.  I have always wanted a simple automated tool to compile all necessary metadata (location, weather, equipment, scientific facts) and processing details into a single, structured document for cataloging my astrophotography sessions.

**My Solution:**
A simple Sequential Agent pipeline that performs end-to-end data collection and artifact generation.

  ***Architecture***

Three specialized agents—LocationAgent, ResearchAgent, and AstroRecordAgent—execute in sequence, sharing state and culminating in a Custom Tool call to generate the final PDF report.

### 💡 Agent Architecture: Sequential Astrophotography Log Pipeline

The system is built as a **Sequential Multi-Agent System** where agents execute in a strict, ordered flow, passing data via shared state management.

| Stage | Agent Name | Primary Tool Used | Output/State Passed |
| :---: | :---: | :---: | :---: |
| **1.** 🌍 | **LocationAgent** | Built-in Tool: Google Search (Weather/Location) | `location_data` |
| **2.** 🔬 | **ResearchAgent** | Built-in Tool: Google Search (Scientific Facts) | `scientific_data` |
| **3.** 📝 | **AstroRecordAgent** | Custom Tool: `process_astro_log` (PDF/Image Processing) | Final PDF Artifact |

---

#### ➡️ System Flow
**User Input** (Target Name + Image) $\rightarrow$ **LocationAgent** $\rightarrow$ **ResearchAgent** $\rightarrow$ **AstroRecordAgent** $\rightarrow$ **Final Catalog PDF**

**Key Concepts**  

This agent solution demonstrates **five key concepts/capabilities**,  of the Agent Development Kit (ADK):

1.  **Multi-agent System (Sequential Agents):** The core architecture is a **Sequential Agent** pipeline, which orchestrates a reliable, ordered workflow between the specialized `LocationAgent`, `ResearchAgent`, and `AstroRecordAgent`.
2.  **Tools (Custom Tool):** A **Custom Tool** (`process_astro_log`) is integrated to handle complex, non-LLM tasks: image processing (`Pillow`) and final PDF catalog generation (`FPDF`), demonstrating integration with external logic.
3.  **Tools (Built-in Tool):** Both the location and research sub-agents effectively leverage the **Built-in Tool: Google Search** to retrieve necessary real-time and scientific data (e.g., weather conditions, astronomical facts).
4.  **Sessions & Memory (State Management):** The system uses explicit **State Management** (`input_key`/`output_key`) to reliably pass and compile the gathered information between the three sequential agents.
5.  **Observability (Tracing):** The execution is fully visible using **Tracing** (`runner.run_debug()`), which is included to show the complete step-by-step logic, tool calls, and state changes for verification.

In [1]:
# @title 1. Install Dependencies and Imports
# We need ADK, FPDF for creating PDFs, Pillow for image processing, and Colab utilities for UI/file upload.
!pip install -q google-adk fpdf pillow

import os
import sys
import asyncio
from datetime import datetime
from PIL import Image, ImageFilter
from fpdf import FPDF
from IPython.display import display
from google.colab import userdata, files

# ADK Imports
from google.genai import types
from google.adk.agents import Agent, SequentialAgent
from google.adk.runners import InMemoryRunner
from google.adk.tools import google_search, FunctionTool

# Global variable to hold user inputs, will be populated by the next cell
USER_INPUT = {}

# Authentication (Colab Specific)
try:
    os.environ["GOOGLE_API_KEY"] = userdata.get('GOOGLE_API_KEY')
    # Explicitly configure the generativeai client with the API key
    import google.generativeai as genai
    genai.configure(api_key=os.environ["GOOGLE_API_KEY"])
    print("✅ API Key loaded and configured.")
except:
    print("❌ Please set 'GOOGLE_API_KEY' in Colab Secrets.")

  Preparing metadata (setup.py) ... done
✅ API Key loaded and configured.


 **2: Interactive Input & File Upload**

This cell demonstrates a built-in form utility to collect dynamic user input: for example: Location where the   Deep space object was photographed, equipment used, day, date etc.  
*n.b: (Final version of the app will have a standalone portal/ui to interface with astrolog app)*

In [2]:

from google.colab import output, files
from IPython.display import display, HTML
import base64
import json

# Global variable to hold user inputs (used in Cell 3)
global USER_INPUT
USER_INPUT = {}

def set_user_inputs(data):
    """
    Python function registered to be called by JavaScript.
    It receives the form data and the base64-encoded file.
    """
    global USER_INPUT

    # Parse the JSON string sent from JavaScript
    form_data = json.loads(data)

    # Extract file data
    image_b64 = form_data.pop('image_data', None)
    image_filename = form_data.pop('image_filename', 'uploaded_image.jpg')

    if image_b64:
        try:
            # Decode the base64 string to bytes
            image_bytes = base64.b64decode(image_b64.split(',')[1])

            # Save the image bytes to a file in the Colab environment
            with open(image_filename, 'wb') as f:
                f.write(image_bytes)

            # --- Time Formatting Logic ---
            # Convert HH:MM (24h format from input type="time") to HH:MM AM/PM
            raw_time = form_data.get("time_str", "00:00")

            # Simple 24-hour to 12-hour conversion logic
            hour, minute = map(int, raw_time.split(':'))
            am_pm = "AM"
            if hour >= 12:
                am_pm = "PM"
            if hour > 12:
                hour -= 12
            if hour == 0:
                hour = 12

            formatted_time = f"{hour:02d}:{minute:02d} {am_pm}"

            # Populate the global input dictionary
            USER_INPUT = {
                "image_path": image_filename,
                "object_name": form_data.get("object_name", ""),
                "date": form_data.get("date_str", ""),
                "time": formatted_time,
                "location": form_data.get("location", ""),
                "equipment": form_data.get("equipment_details", "No equipment details provided.") # New field
            }

            print("\n✅ Inputs and Image Captured!")
            print(f"- Object: {USER_INPUT['object_name']}")
            print(f"- Location: {USER_INPUT['location']}")
            print(f"- Date/Time: {USER_INPUT['date']} at {USER_INPUT['time']}")
            print(f"- Equipment: {USER_INPUT['equipment'][:50]}...") # Show snippet
            print(f"- Image File: {USER_INPUT['image_path']}")
            print("\nReady to run the multi-agent pipeline (Cell 3).")

        except Exception as e:
            print(f"❌ Error processing input data or image upload: {e}")

    else:
        print("❌ ERROR: No image file was submitted.")

# Register the Python function handler
output.register_callback('set_user_inputs', set_user_inputs)

# HTML/JavaScript for the custom form
form_html = """
<div style="border: 2px solid #4CAF50; padding: 20px; border-radius: 10px; background-color: #f9f9f9;">
    <h3 style="color: #333; margin-top: 0;">📸 Astrophotography Session Logger Input</h3>
    <form id="astroForm">
        <label for="object_name" style="font-weight: bold;">Deep Space Object Name:</label><br>
        <input type="text" id="object_name" name="object_name" value="Whirlpool Galaxy" style="width: 90%; padding: 8px; margin-bottom: 10px; border: 1px solid #ccc; border-radius: 4px;" required><br>

        <label for="location" style="font-weight: bold;">Location Captured:</label><br>
        <input type="text" id="location" name="location" value="Union City, California" style="width: 90%; padding: 8px; margin-bottom: 10px; border: 1px solid #ccc; border-radius: 4px;" required><br>

        <label for="date_str" style="font-weight: bold;">Date:</label><br>
        <input type="date" id="date_str" name="date_str" value="2025-11-15" style="width: 90%; padding: 8px; margin-bottom: 10px; border: 1px solid #ccc; border-radius: 4px;" required><br>

        <label for="time_str" style="font-weight: bold;">Time:</label><br>
        <input type="time" id="time_str" name="time_str" value="22:45" style="width: 90%; padding: 8px; margin-bottom: 10px; border: 1px solid #ccc; border-radius: 4px;" required><br>

        <hr style="margin: 15px 0;">

        <label for="equipment_details" style="font-weight: bold;">Equipment Details:</label><br>
        <textarea id="equipment_details" name="equipment_details" rows="4" style="width: 90%; padding: 8px; margin-bottom: 10px; border: 1px solid #ccc; border-radius: 4px;">Telescope: 8" Dobsonian; Mount: Sky-Watcher EQ6-R Pro; Camera: ZWO ASI1600MM Pro; Filter: Ha (7nm).</textarea><br>

        <hr style="margin: 15px 0;">

        <label for="image_file" style="font-weight: bold;">Upload Photograph (JPEG/PNG):</label><br>
        <input type="file" id="image_file" name="image_file" accept="image/jpeg, image/png" style="margin-bottom: 15px;" required><br>

        <button type="submit" style="background-color: #4CAF50; color: white; padding: 10px 15px; border: none; border-radius: 4px; cursor: pointer;">
            🚀 Run Astro Log Data Pipeline
        </button>
        <p id="status" style="color: red; margin-top: 10px;"></p>
    </form>
</div>

<script>
    document.getElementById('astroForm').addEventListener('submit', function(e) {
        e.preventDefault();

        const status = document.getElementById('status');
        const fileInput = document.getElementById('image_file');
        const file = fileInput.files[0];

        if (!file) {
            status.textContent = 'Please upload your Astro-photograph.';
            return;
        }

        status.textContent = 'Processing and sending data to Python...';

        const reader = new FileReader();
        reader.onload = function(event) {
            const file_data = event.target.result;

            // Collect all form data, including the equipment field
            const formData = {
                object_name: document.getElementById('object_name').value,
                location: document.getElementById('location').value,
                date_str: document.getElementById('date_str').value,
                time_str: document.getElementById('time_str').value,
                equipment_details: document.getElementById('equipment_details').value, // NEW
                image_data: file_data,
                image_filename: file.name
            };

            // Call the registered Python function with the JSON data
            google.colab.kernel.invokeFunction('set_user_inputs', [JSON.stringify(formData)], {});
            status.textContent = 'Data sent. Check the output below for confirmation.';
        };
        reader.readAsDataURL(file);
    });
</script>
"""

# Display the custom form in the notebook
display(HTML(form_html))



✅ Inputs and Image Captured!
- Object: Whirlpool Galaxy
- Location: Union City, California
- Date/Time: 2025-11-15 at 10:45 PM
- Equipment: Telescope: 8" Dobsonian; Mount: Sky-Watcher EQ6-R ...
- Image File: Whirlpool.Galaxy.M51.jpg

Ready to run the multi-agent pipeline (Cell 3).


## 3. Define the AstroLog Generation Tool

The function, `process_astro_log`, acts as the final step in the sequential pipeline. It takes the processed image path and text outputs from the preceding agents, enhances the image (sharpening and scaling), and uses all the information to generate a comprehensive, structured PDF log file.

In [3]:
def process_astro_log(image_path: str, object_name: str, research_text: str, location_text: str) -> str:
    """
    Sharpen the image, proportionally resize it to fit on the first page, and generate a PDF log book entry.
    """
    try:
        # 1. Image Processing (Sharpening and Scaling)
        print(f"⚙️ Sharpening and Scaling image: {image_path}...")

        original_image = Image.open(image_path)
        sharpened_image = original_image.filter(ImageFilter.SHARPEN)

        # --- Image Scaling Logic ---
        # Standard A4 page is 210mm wide. We use 180mm for the max width (15mm margins).
        MAX_WIDTH_MM = 180
        # A safe max height to leave room for the title and the text blocks on page 1
        MAX_HEIGHT_MM = 110 # Reduced slightly to fit Location, Equipment, and Research start on Page 1

        original_width_px, original_height_px = sharpened_image.size
        ratio = original_width_px / original_height_px

        target_width_mm = MAX_WIDTH_MM
        target_height_mm = MAX_WIDTH_MM / ratio

        # If the calculated height is too tall, scale based on MAX_HEIGHT_MM instead
        if target_height_mm > MAX_HEIGHT_MM:
            target_height_mm = MAX_HEIGHT_MM
            target_width_mm = MAX_HEIGHT_MM * ratio

            if target_width_mm > MAX_WIDTH_MM:
                 target_width_mm = MAX_WIDTH_MM
                 target_height_mm = MAX_WIDTH_MM / ratio

        processed_img_path = "processed_astro.png"
        sharpened_image.save(processed_img_path)

        # 2. PDF Generation
        print("📄 Generating PDF Report...")
        pdf = FPDF()
        pdf.add_page()

        # --- Title ---
        pdf.set_font("Arial", 'B', 24)
        pdf.cell(0, 20, f"Astro Log: {object_name}", ln=True, align='C')

        # --- Add Image (Dynamically Sized) ---
        x_pos = (pdf.w - target_width_mm) / 2

        pdf.image(processed_img_path, x=x_pos, w=target_width_mm, h=target_height_mm)
        pdf.ln(10) # Line break after the image

        # --- Location & Weather Info ---
        pdf.set_font("Arial", 'B', 14)
        pdf.cell(0, 10, "Session Details:", ln=True)
        pdf.set_font("Arial", '', 11)
        pdf.multi_cell(0, 6, location_text.encode('latin-1', 'replace').decode('latin-1'))
        pdf.ln(5)

        # --- Equipment Info (NEW SECTION) ---
        pdf.set_font("Arial", 'B', 14)
        pdf.cell(0, 10, "Equipment Used:", ln=True)
        pdf.set_font("Arial", '', 11)
        # Access the equipment details from the global USER_INPUT dictionary
        equipment_details = USER_INPUT.get('equipment', 'No equipment details provided.')
        pdf.multi_cell(0, 6, equipment_details.encode('latin-1', 'replace').decode('latin-1'))
        pdf.ln(5)

        # --- Research Info ---
        pdf.set_font("Arial", 'B', 14)
        pdf.cell(0, 10, "Object Research:", ln=True)
        pdf.set_font("Arial", '', 11)
        pdf.multi_cell(0, 6, research_text.encode('latin-1', 'replace').decode('latin-1'))

        output_filename = f"AstroLog_{object_name.replace(' ', '_')}.pdf"
        pdf.output(output_filename)
        os.remove(processed_img_path)

        return f"SUCCESS: Log generated: {output_filename}"

    except Exception as e:
        return f"ERROR: Failed to generate log. Details: {str(e)}"

In [4]:
# @title 4. Define Agents and Run the Application

# Make sure all required libraries for the custom tool are imported (assumed in Cell 1)
from PIL import Image, ImageFilter
from fpdf import FPDF
import os
import sys
import asyncio
from google.adk.agents import Agent, SequentialAgent
from google.adk.runners import InMemoryRunner
from google.adk.tools import google_search, FunctionTool

# Global variable USER_INPUT is populated in Cell 2

if not USER_INPUT or not USER_INPUT.get('image_path'):
    print("🔴 ERROR: User inputs were not collected. Please fill out the form in Cell 2 and click 'Run Astro Log Pipeline' before running this cell.")
else:
    # --- Custom Tool Definition (MODIFIED FOR IMAGE SCALING & EQUIPMENT) ---
    def process_astro_log(image_path: str, object_name: str, research_text: str, location_text: str) -> str:
        """
        Sharpen the image, proportionally resize it to fit on the first page, and generate a PDF log book entry.
        """
        try:
            # 1. Image Processing (Sharpening and Scaling)
            print(f"⚙️ Sharpening and Scaling image: {image_path}...")

            original_image = Image.open(image_path)
            sharpened_image = original_image.filter(ImageFilter.SHARPEN)

            # --- Image Scaling Logic ---
            # Standard A4 page is 210mm wide. We use 180mm for the max width (15mm margins).
            MAX_WIDTH_MM = 180
            # A safe max height to leave room for the title and the text blocks on page 1
            MAX_HEIGHT_MM = 110 # Reduced slightly to fit Location, Equipment, and Research start on Page 1

            original_width_px, original_height_px = sharpened_image.size
            ratio = original_width_px / original_height_px

            target_width_mm = MAX_WIDTH_MM
            target_height_mm = MAX_WIDTH_MM / ratio

            # If the calculated height is too tall, scale based on MAX_HEIGHT_MM instead
            if target_height_mm > MAX_HEIGHT_MM:
                target_height_mm = MAX_HEIGHT_MM
                target_width_mm = MAX_HEIGHT_MM * ratio

                if target_width_mm > MAX_WIDTH_MM:
                     target_width_mm = MAX_WIDTH_MM
                     target_height_mm = MAX_WIDTH_MM / ratio

            processed_img_path = "processed_astro.png"
            sharpened_image.save(processed_img_path)

            # 2. PDF Generation
            print("📄 Generating PDF Report...")
            pdf = FPDF()
            pdf.add_page()

            # --- Title ---
            pdf.set_font("Arial", 'B', 24)
            pdf.cell(0, 20, f"Astro Log: {object_name}", ln=True, align='C')

            # --- Add Image (Dynamically Sized) ---
            x_pos = (pdf.w - target_width_mm) / 2

            pdf.image(processed_img_path, x=x_pos, w=target_width_mm, h=target_height_mm)
            pdf.ln(10) # Line break after the image

            # --- Location & Weather Info ---
            pdf.set_font("Arial", 'B', 14)
            pdf.cell(0, 10, "Session Details:", ln=True)
            pdf.set_font("Arial", '', 11)
            pdf.multi_cell(0, 6, location_text.encode('latin-1', 'replace').decode('latin-1'))
            pdf.ln(5)

            # --- Equipment Info (NEW SECTION) ---
            pdf.set_font("Arial", 'B', 14)
            pdf.cell(0, 10, "Equipment Used:", ln=True)
            pdf.set_font("Arial", '', 11)
            # Access the equipment details from the global USER_INPUT dictionary
            equipment_details = USER_INPUT.get('equipment', 'No equipment details provided.')
            pdf.multi_cell(0, 6, equipment_details.encode('latin-1', 'replace').decode('latin-1'))
            pdf.ln(5)

            # --- Research Info ---
            pdf.set_font("Arial", 'B', 14)
            pdf.cell(0, 10, "Object Research:", ln=True)
            pdf.set_font("Arial", '', 11)
            pdf.multi_cell(0, 6, research_text.encode('latin-1', 'replace').decode('latin-1'))

            output_filename = f"AstroLog_{object_name.replace(' ', '_')}.pdf"
            pdf.output(output_filename)
            os.remove(processed_img_path)

            return f"SUCCESS: Log generated: {output_filename}"

        except Exception as e:
            return f"ERROR: Failed to generate log. Details: {str(e)}"

    astro_tool = FunctionTool(process_astro_log)

    # --- Agent 1: Location & Weather Agent ---
    location_agent = Agent(
        name="LocationAgent",
        model="gemini-2.5-flash-lite",
        instruction=f"""
        You are an expert meteorologist and geographer.
        Find the geographic coordinates and the weather conditions (cloud cover, seeing, moon phase)
        for this location: '{USER_INPUT['location']}' on this date: '{USER_INPUT['date']}' at '{USER_INPUT['time']}'.
        Use google_search. Summarize this into a concise, **single plain-text paragraph** (max 100 words).
        **Do not use any markdown headers, lists, or bolding.**
        """,
        tools=[google_search],
        output_key="location_data"
    )

    # --- Agent 2: Research Agent ---
    research_agent = Agent(
        name="ResearchAgent",
        model="gemini-2.5-flash-lite",
        instruction=f"""
        You are an expert astrophysicist.
        Research the Deep Space Object: '{USER_INPUT['object_name']}'.
        Find 3-4 interesting scientific facts about it (Distance, Magnitude, Composition, etc.).
        Summarize this into a concise, **single plain-text paragraph** (max 150 words).
        **CRITICAL: Your entire output MUST be only this single plain-text paragraph. Do not use any markdown formatting (like bolding, lists, or headers) or conversational greetings.**
        """,
        tools=[google_search],
        output_key="scientific_data"
    )

    # --- Agent 3: AstroRecord Agent ---
    record_agent = Agent(
        name="AstroRecordAgent",
        model="gemini-2.5-flash-lite",
        instruction=f"""
        You are the final publisher. Your SOLE task is to call the 'process_astro_log' tool.
        DO NOT provide any other text, explanation, or conversational filler before or after the tool call.

        Action:
        Call the 'process_astro_log' tool with the following arguments:
        - image_path: '{USER_INPUT['image_path']}'
        - object_name: '{USER_INPUT['object_name']}'
        - research_text: {{scientific_data}}
        - location_text: {{location_data}}
        """,
        tools=[astro_tool]
    )

    # --- Root Agent: The Orchestrator (Using SequentialAgent with descriptive fields) ---
    root_agent = SequentialAgent(
        name="AstrophotographyFieldCatalog", # Corrected name
        sub_agents=[location_agent, research_agent, record_agent]
    )

    print("✅ All Agents defined.")

    # --- Execution ---
    async def main():
        print(f"\n🚀 Starting Astrophotography Pipeline for {USER_INPUT['object_name']}...")
        print("-" * 60)

        runner = InMemoryRunner(agent=root_agent)

        # Await the run_debug call to get the complete trace result
        full_trace_result = await runner.run_debug("Create an astrophotography log.")

        # Based on the AttributeError, full_trace_result itself is a list of trace events.
        response_history = full_trace_result

        # Print all events from the collected history
        for i, event in enumerate(response_history):
            print(f"[COLLECTED EVENT {i+1}]: {event}")

        # The final event is still the last one in the history
        final_event = response_history[-1] if response_history else None

        if final_event and hasattr(final_event, 'output'):
            final_output = str(final_event.output)
        else:
            final_output = "Tool executed, final agent output was suppressed or no final event." # More descriptive
            if final_event and hasattr(final_event, 'content'):
                 final_output = str(final_event.content)

        expected_file = f"AstroLog_{USER_INPUT['object_name'].replace(' ', '_')}.pdf"

        print("\n" + "="*60)

        # Check for file existence (most reliable success check)
        if os.path.exists(expected_file):
            print(f"🎉 PIPELINE SUCCESS! File created: {expected_file}")
            print("The PDF was generated successfully, and the image is now scaled to fit the first page.")
            print(f"\n--- RAW ADK DEBUG OUTPUT ---\n{final_output}")
            print("Please check the 'Files' tab (folder icon on the left sidebar) to download your PDF.")
        else:
            print(f"❌ PIPELINE FAILED. Final Output: {final_output}")

        print("\n--- FULL ADK RESPONSE HISTORY (FOR DETAILED TRACING) ---")
        for i, event in enumerate(response_history):
            print(f"Event {i}: {event}")

        print("="*60)

    # Run the async loop in Colab
    await main()

✅ All Agents defined.

🚀 Starting Astrophotography Pipeline for Whirlpool Galaxy...
------------------------------------------------------------

 ### Created new session: debug_session_id

User > Create an astrophotography log.
LocationAgent > Union City, California is located at approximately 37.59° N latitude and -122.02° W longitude. On the evening of November 15, 2025, at 10:45 PM, the weather in Union City is expected to be clear with temperatures around 7°C (45°F). There is a 20% chance of precipitation, and the moon will be in a waxing gibbous phase, approximately 78% illuminated.
ResearchAgent > The Whirlpool Galaxy, also known as M51, is approximately 31 million light-years away from Earth. It has an apparent magnitude of 8.4, making it visible through small telescopes under dark skies. This grand-design spiral galaxy is characterized by its prominent, well-defined spiral arms, which are active regions of star formation. The Whirlpool Galaxy's striking appearance is significa

⚙️ Sharpening and Scaling image: Whirlpool.Galaxy.M51.jpg...
📄 Generating PDF Report...
[COLLECTED EVENT 1]: model_version='gemini-2.5-flash-lite' content=Content(
  parts=[
    Part(
      text='Union City, California is located at approximately 37.59° N latitude and -122.02° W longitude. On the evening of November 15, 2025, at 10:45 PM, the weather in Union City is expected to be clear with temperatures around 7°C (45°F). There is a 20% chance of precipitation, and the moon will be in a waxing gibbous phase, approximately 78% illuminated.'
    ),
  ],
  role='model'
) grounding_metadata=GroundingMetadata(
  grounding_chunks=[
    GroundingChunk(
      web=GroundingChunkWeb(
        title='latitude.to',
        uri='https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQFKmy4bV_xQwAD8_Np5wXCx8jYSHj1-xQfSIFgrJXE1LUYt6YGf_qJof2i84DBk0ocUK9viDr-32vsRNk50zf8yIgYeVgNxo-PeQF2Nay3cAzdq4oo3KlyyNwIdJtuLoP_L5UiOKOWGZu9nt2Imy0i_306SwMYQDSPVNA6YabokmA=='
      )
    ),
    Grounding


## Summary: Astrophotography Log Analysis

The multi-agent pipeline executed successfully to create a comprehensive astrophotography log for the **Whirlpool Galaxy (M51)**.

1.  **Location and Weather Data:** The **`LocationAgent`** retrieved the geographic coordinates and expected weather conditions (partly cloudy, $50^{\circ}\text{F}$, waxing gibbous moon on November 15, 2025) for Union City, California.
2.  **Scientific Research:** The **`ResearchAgent`** gathered detailed facts about the target, including its distance (31 million light-years), apparent magnitude (8.4), and key characteristics as a grand-design spiral galaxy.
3.  **Log Generation:** Finally, the **`AstroRecordAgent`** utilized the collected data and the input image (`Whirlpool.Galaxy.M51.jpg`) by invoking the `process_astro_log` tool. This function successfully processed the image (sharpening/scaling) and compiled all information into the final report.

The pipeline completed successfully, producing the artifact: **`AstroLog_Whirlpool_Galaxy.pdf`**.